In [213]:
# Packages needed
import pandas as pd
import numpy as np
import os
import itertools
import pycountry

In [215]:
# working directory
os.chdir("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged")


In [216]:
# Loading all sheets into a dictionary
df_dict = pd.read_excel("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/dataset.xlsx", sheet_name=None)

# Checking sheet dummylanguage
print(df_dict['Dummy_language'].head())


   Unnamed: 0 iso3_o    country_o iso3_d country_d  col  csl  cnl  prox1  \
0           0    AFG  Afghanistan    ALB   Albania    0  0.0  0.0   0.25   
1           1    AFG  Afghanistan    DZA   Algeria    0  0.0  0.0   0.00   
2           2    AFG  Afghanistan    AND   Andorra    0  0.0  0.0   0.25   
3           3    AFG  Afghanistan    AGO    Angola    0  0.0  0.0   0.25   
4           4    AFG  Afghanistan    AIA  Anguilla    0  0.0  0.0   0.25   

          lp1     prox2           lp2        cl       cle  
0         NaN  0.073421           NaN       NaN  0.086628  
1         0.0  0.085278  7.640970e-01  0.099927  0.100619  
2  19458665.0  0.120260  1.077538e+07  0.140918  0.141894  
3         NaN  0.118905           NaN       NaN  0.140295  
4         NaN  0.092914           NaN       NaN  0.109629  


In [217]:
# Total World Exports per year(Worldbank)

df_totworldexp = pd.read_excel('/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/total_world_exports.xlsx')

# Importing World Population,GDP and if a country is landlocked(dummy)
gdp_world = pd.read_excel("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/gdp_world.xlsx")
pop_world = pd.read_excel("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/population_world.xlsx")
landlocked = pd.read_excel("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/landlocked-countries-2025.xlsx")

In [218]:
# CEPII dataset
df_cepii = pd.read_csv('/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/Gravity_V202211.csv', low_memory=False)
df_cepii.head()

,year,country_id_o,country_id_d,iso3_o,iso3_d,iso3num_o,iso3num_d,country_exists_o,country_exists_d,gmt_offset_2020_o,...,entry_time_o,entry_time_d,entry_tp_o,entry_tp_d,tradeflow_comtrade_o,tradeflow_comtrade_d,tradeflow_baci,manuf_tradeflow_baci,tradeflow_imf_o,tradeflow_imf_d
0,1948,ABW,ABW,ABW,ABW,533.0,533.0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1949,ABW,ABW,ABW,ABW,533.0,533.0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950,ABW,ABW,ABW,ABW,533.0,533.0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1951,ABW,ABW,ABW,ABW,533.0,533.0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1952,ABW,ABW,ABW,ABW,533.0,533.0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
# Load all sheets
df_dict = pd.read_excel("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/dataset.xlsx", sheet_name=None)

# Extracting individual sheets
cleaned = df_dict["cleaned_dataset"]
gdpop = df_dict["GDP_POP"]
dlang = df_dict["Dummy_language"]

# Merge GDP variables for origin countries
cleaned = cleaned.merge(
    gdpop[["iso3_o", "Time",
           "GDP (current US$) [NY.GDP.MKTP.CD]",
           "GDP per capita (current US$) [NY.GDP.PCAP.CD]",
           "Population, total [SP.POP.TOTL]"]],
    left_on=["iso3_o", "year"],
    right_on=["iso3_o", "Time"],
    how="left"
)

# Rename for origin clarity
cleaned.rename(columns={
    "GDP (current US$) [NY.GDP.MKTP.CD]": "GDP_o",
    "GDP per capita (current US$) [NY.GDP.PCAP.CD]": "GDPpc_o",
    "Population, total [SP.POP.TOTL]": "Pop_o"
}, inplace=True)

# Merge GDP variables for destination countries
cleaned = cleaned.merge(
    gdpop[["iso3_d", "Time",
           "GDP (current US$) [NY.GDP.MKTP.CD]",
           "GDP per capita (current US$) [NY.GDP.PCAP.CD]",
           "Population, total [SP.POP.TOTL]"]],
    left_on=["iso3_d", "year"],
    right_on=["iso3_d", "Time"],
    how="left"
)

# Rename for destination clarity
cleaned.rename(columns={
    "GDP (current US$) [NY.GDP.MKTP.CD]": "GDP_d",
    "GDP per capita (current US$) [NY.GDP.PCAP.CD]": "GDPpc_d",
    "Population, total [SP.POP.TOTL]": "Pop_d"
}, inplace=True)

# Variable NATO(binary)
nato_iso = [
    'ALB', 'BEL', 'BGR', 'CAN', 'HRV', 'CZE', 'DNK', 'EST', 'FIN', 'FRA',
    'DEU', 'GRC', 'HUN', 'ISL', 'ITA', 'LVA', 'LTU', 'LUX', 'MNE', 'NLD',
    'MKD', 'NOR', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'TUR',
    'GBR', 'USA'
]

# Create dummy variable: 1 if iso3_o is a NATO member, 0 otherwise
cleaned["NATO_o"] = cleaned["iso3_o"].isin(nato_iso).astype(int)
cleaned["NATO_d"] = cleaned["iso3_d"].isin(nato_iso).astype(int)
cleaned["NATO_d"].head()

# Make sure dlang has matching keys (e.g., iso3_o, iso3_d)
cleaned = cleaned.merge(
    dlang,
    on=["iso3_o", "iso3_d"],
    how="left"
)

# Saving dataset
#cleaned.to_excel("dataset1.xlsx", index=False)
df_cleaned = cleaned

In [220]:
# inspecting merge
df_cleaned.head()

,country_o_x,iso3_o,country_d_x,iso3_d,year,exports,imports,Time_x,GDP_o,GDPpc_o,...,country_d_y,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
0,Austria,AUT,Belgium,BEL,1999,9.851537e+08,1.654605e+09,1999,216421809125.915527,27078.708161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Austria,AUT,Belgium,BEL,2000,1.011753e+09,1.803905e+09,2000,196181599831.286682,24487.297469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Austria,AUT,Belgium,BEL,2001,1.225178e+09,1.848592e+09,2001,196477206829.33017,24430.495983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Austria,AUT,Belgium,BEL,2002,1.249067e+09,2.001089e+09,2002,212837224022.813324,26334.862215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Austria,AUT,Belgium,BEL,2003,1.561746e+09,2.378743e+09,2003,260779834340.09671,32110.115966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
# Merge with CEPII dataset

df_merged = pd.merge(
    df_cleaned,
    df_cepii,
    left_on=["iso3_o", "iso3_d", "year"],
    right_on=["iso3_o", "iso3_d", "year"],
    how="left"
)


In [222]:
# Checking columns after merge
df_merged.head()

,country_o_x,iso3_o,country_d_x,iso3_d,year,exports,imports,Time_x,GDP_o,GDPpc_o,...,entry_time_o,entry_time_d,entry_tp_o,entry_tp_d,tradeflow_comtrade_o,tradeflow_comtrade_d,tradeflow_baci,manuf_tradeflow_baci,tradeflow_imf_o,tradeflow_imf_d
0,Austria,AUT,Belgium,BEL,1999,9.851537e+08,1.654605e+09,1999,216421809125.915527,27078.708161,...,NaN,NaN,NaN,NaN,851191.052,1112586.330,994049.213,982682.313,985153.750,1083212.625
1,Austria,AUT,Belgium,BEL,2000,1.011753e+09,1.803905e+09,2000,196181599831.286682,24487.297469,...,NaN,NaN,NaN,NaN,918013.097,1098237.551,1023440.338,1008678.188,1011752.688,1096356.125
2,Austria,AUT,Belgium,BEL,2001,1.225178e+09,1.848592e+09,2001,196477206829.33017,24430.495983,...,NaN,NaN,NaN,NaN,1111910.584,1235302.981,1202675.553,1174673.125,1225177.625,1234874.250
3,Austria,AUT,Belgium,BEL,2002,1.249067e+09,2.001089e+09,2002,212837224022.813324,26334.862215,...,NaN,NaN,NaN,NaN,1121961.226,1022840.482,1236554.695,1163574.750,1249067.250,1025236.750
4,Austria,AUT,Belgium,BEL,2003,1.561746e+09,2.378743e+09,2003,260779834340.09671,32110.115966,...,25.0,56.0,33.0,63.0,1394691.127,1296890.503,1434758.711,1388038.875,1561746.250,1301145.375


In [223]:
# -----------------------------
# 1. Loading GSDB Sanctions Dataset
# -----------------------------
df_s = pd.read_csv("/Users/omid/Desktop/School/Thesis_STREEM/Data/Merged/GSDB_V4.csv", low_memory=False)

# -----------------------------
# 2. Define ISO3 Conversion with Manual Overrides
# -----------------------------
def get_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        manual_map = {
            "German Democratic Republic": "DDR",     # East Germany
            "Soviet Union": "SUN",
            "Yugoslavia": "YUG",
            "Czechoslovakia": "CSK",
            "CoCom": None,
            "Organization of American States": None,
            "Palestinian Authority": "PSE",
            "Serbia and Montenegro": "SCG",
            "North Korea": "PRK"
        }
        return manual_map.get(name, None)

# -----------------------------
# 3. Expand Sanctions Over Years
# -----------------------------
expanded_rows = df_s.apply(lambda row: [
    {
        "year": year,
        "sanctioning_state": row["sanctioning_state"],
        "sanctioned_state": row["sanctioned_state"],
        "trade": row["trade"],
        "descr_trade": row["descr_trade"],
        "financial": row["financial"],
        "success": row["success"]
    }
    for year in range(int(row["begin"]), int(row["end"]) + 1)
], axis=1)

df_s_expanded = pd.DataFrame(itertools.chain.from_iterable(expanded_rows))

# -----------------------------
# 4. Extracting ISO3 codes
# -----------------------------
df_s_expanded["iso3_o"] = df_s_expanded["sanctioning_state"].apply(get_iso3)
df_s_expanded["iso3_d"] = df_s_expanded["sanctioned_state"].apply(get_iso3)

# -----------------------------
# 5. Creating subset data
# -----------------------------
sanctions_data = df_s_expanded[[
    "year", "iso3_o", "iso3_d", "trade", "descr_trade", "financial", "success"
]].copy()

sanctions_data["sanction_active"] = sanctions_data["trade"].apply(lambda x: 1 if x == 1 else 0)

# -----------------------------
# 6. Final merge with previous data
# -----------------------------
sanctions_data_renamed = sanctions_data.rename(columns={
    'trade': 'sanction_trade',
    'descr_trade': 'sanction_descr_trade',
    'success': 'sanction_success',
    'financial': 'sanction_financial'
})

df_merged = pd.merge(
    df_merged,
    sanctions_data_renamed,
    on=["year", "iso3_o", "iso3_d"],
    how="left"
)



In [224]:
# Adding total worlds exports to my existing df
df_merged = pd.merge(df_merged, df_totworldexp, on='year', how='left')
df_merged.rename(columns={'export': 'world_exports'}, inplace=True)

In [225]:
# Final merge
df_merged = (
    df_merged
    .merge(gdp_world[['year', 'world_gdp']], on='year', how='left')
    .merge(pop_world[['year', 'world_population']], on='year', how='left')
    .merge(
        landlocked[['iso3_o', 'landlocked']].rename(columns={'landlocked': 'landlocked_o'}),
        on='iso3_o', how='left'
    )
    .merge(
        landlocked[['iso3_d', 'landlocked']].rename(columns={'landlocked': 'landlocked_d'}),
        on='iso3_d', how='left'
    )
)


In [226]:
# Save to Excel for R
df_merged.to_excel("df_merged.xlsx", index=False)